In [0]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [0]:
import pandas as pd
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [0]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
)

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           transform=transform,download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           transform=transform)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
num_epochs = 5
batch_size = 8
learning_rate = 0.001

In [0]:
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [8]:
examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

torch.Size([8, 3, 32, 32]) torch.Size([8])


In [0]:
classes = ('plane', 'car', 'bird' ,'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)   # (32 - 5 + 2 * 0) / 1 + 1 = 28
        self.pool = nn.MaxPool2d(2,2)     # 28 / 2 = 14
        self.conv2 = nn.Conv2d(6, 16, 5)  # (14 - 5 + 2 * 0) / 1 + 1 = 10
        self.fc1 = nn.Linear(16*5*5, 120) # 10 / 2 = 5
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [12]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoch : {epoch+1} / {num_epochs},\
            Step : {i+1} / {total_step}, Loss : {loss.item():.4f}')

Epoch : 1 / 5,            Step : 2000 / 6250, Loss : 2.2978
Epoch : 1 / 5,            Step : 4000 / 6250, Loss : 2.2873
Epoch : 1 / 5,            Step : 6000 / 6250, Loss : 2.2907
Epoch : 2 / 5,            Step : 2000 / 6250, Loss : 2.2588
Epoch : 2 / 5,            Step : 4000 / 6250, Loss : 2.2374
Epoch : 2 / 5,            Step : 6000 / 6250, Loss : 2.0119
Epoch : 3 / 5,            Step : 2000 / 6250, Loss : 1.6499
Epoch : 3 / 5,            Step : 4000 / 6250, Loss : 1.9694
Epoch : 3 / 5,            Step : 6000 / 6250, Loss : 1.5958
Epoch : 4 / 5,            Step : 2000 / 6250, Loss : 2.3195
Epoch : 4 / 5,            Step : 4000 / 6250, Loss : 2.0188
Epoch : 4 / 5,            Step : 6000 / 6250, Loss : 1.8098
Epoch : 5 / 5,            Step : 2000 / 6250, Loss : 1.5482
Epoch : 5 / 5,            Step : 4000 / 6250, Loss : 1.8535
Epoch : 5 / 5,            Step : 6000 / 6250, Loss : 1.4708


In [14]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted==labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'acc : {acc}')

acc : 40.43
